How to deside number of Hideen layers and Neuron in hidden layer?
By using keras hyper-tuning

In [70]:

import pandas as pd
from keras  import layers
from keras.models import Sequential
from keras.layers import Dense
from kerastuner.tuners import RandomSearch

In [71]:
df = pd.read_csv("Real_Combine.csv")

In [72]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1093 non-null   float64
 1   TM      1093 non-null   float64
 2   Tm      1093 non-null   float64
 3   SLP     1093 non-null   float64
 4   H       1093 non-null   float64
 5   VV      1093 non-null   float64
 6   V       1093 non-null   float64
 7   VM      1093 non-null   float64
 8   PM 2.5  1092 non-null   float64
dtypes: float64(9)
memory usage: 77.0 KB


In [74]:
df.isnull().values.all()

False

In [75]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

## Hyperparameters

How many number of hidden layers we should have?

How many number of neurons we should have in hidden layers?

Learning Rate

In [76]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [77]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')


Reloading Tuner from project\Air Quality Index\tuner0.json


In [78]:
tuner.search_space_summary()

Search space summary
Default search space size: 17
number of layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
unite0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.1, 0.001, 0.0001], 'ordered': True}
unite1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
un

In [79]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.30,random_state=42)

In [82]:
try:
        tuner.search(x_train, y_train,epochs=5,
                validation_data=(x_test, y_test))
except RuntimeError:
    print("RuntimeError")

Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 77.1090 - mean_absolute_error: 77.1090 - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 68.8705 - mean_absolute_error: 68.8705 - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 63.5934 - mean_absolute_error: 63.5934 - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 70.1517 - mean_absolute_error: 70.1517 - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 67.3982 - mean_absolute_error: 67.3982 - val_loss: nan - val_mean_absolute_error: nan
Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 145.1927 - mean_absolute_error: 145.1927 - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 95.6055 - mean_absolute_error: 95.6055 - val_loss: nan - val_mean_absolute_error:

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\metrics_tracking.py:111: RuntimeWarning: All-NaN axis encountered
  np.nanmin(values) if self.direction == "min" else np.nanmax(values)


In [83]:
tuner.search_space_summary()

Search space summary
Default search space size: 21
number of layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
unite0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.1, 0.001, 0.0001], 'ordered': True}
unite1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
unite5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
un